# MNIST Handwritted Digit Recognizer

### Import statements

In [ ]:
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import cv2
from PIL import Image

## Training On MNIST data

### Function to load Dataset

In [ ]:
def load_dataset():
    (trainX,trainY),(testX,testY) = mnist.load_data()
    trainX = trainX.reshape((trainX.shape[0],28,28,1))
    testX = testX.reshape((testX.shape[0],28,28,1))
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX,trainY,testX,testY

### Function to normalize the images

In [ ]:
def prep_pixels(train,test):
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    train_norm = train_norm/255.0
    test_norm = test_norm/255.0
    return train_norm,test_norm

### Model Defination

In [ ]:
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax')) 
    opt = SGD(lr=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

### Evaluate Model Defined

In [ ]:
def evaluate_model(dataX, dataY, n_folds=5):
    scores, histories = list(), list()
    # prepare cross validation
    kfold = KFold(n_folds, shuffle=True, random_state=1)
    # enumerate splits
    for train_ix, test_ix in kfold.split(dataX):
        # define model
        model = define_model()
        # select rows for train and test
        trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
        # fit model
        history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
        # evaluate model
        _, acc = model.evaluate(testX, testY, verbose=0)
        print('> %.3f' % (acc * 100.0))
        # stores scores
        scores.append(acc)
        histories.append(history)
        return scores, histories

### Train the model

In [ ]:
# run the test harness for evaluating a model
def run_test_harness():
    # load dataset
    trainX, trainY, testX, testY = load_dataset()
    # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)
    # define model
    model = define_model()
    # fit model
    model.fit(trainX, trainY, epochs = 10, batch_size=32, verbose=0)
    # save model
    model.save('final_model.h5')
    _, acc = model.evaluate(testX, testY, verbose=0)
    print('> %.3f' % (acc * 100.0))

In [ ]:
run_test_harness()    #training the model

## Making Predictions on custom Images

### Testing on real image

In [ ]:
def run_live_example():
    
    #read the image. webcam module can also be added
    image = cv2.imread('3_1.jpg')
    
    #convert image into grayscale for thresholding
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #threshold the image with inverse flag for binarization
    ret,thresh_image = cv2.threshold(image,95,255,cv2.THRESH_BINARY_INV)
    
    #display the image
    cv2.imshow('thresh_image',thresh_image)
    cv2.waitKey(0)
    
    #resithe image as per the model requirement
    resized_thresh_image = cv2.resize(thresh_image,(28,28))
    
    resized_thresh_image = resized_thresh_image.reshape(1, 28, 28, 1)
    
    resized_thresh_image = resized_thresh_image.astype('float32')
    
    resized_thresh_image = resized_thresh_image / 255.0
    
    #load the model
    model = load_model('final_model.h5')
    
    #prediction
    digit = model.predict_classes(resized_thresh_image)
    
    print(digit[0])
    
    cv2.destroyAllWindows()


### Running Function to test

In [ ]:
run_live_example()